In [1]:
%whos

Interactive namespace is empty.


In [1]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
# from imblearn.over_sampling import BorderlineSMOTE
import random
np.random.seed(42)
tf.set_random_seed(42)
random.seed(42)

In [2]:
np_Images = np.load('./dataset/Image_224.npy') #converting the list of images to numpy array.
np_Labels = np.load('./dataset/Label_224.npy')

In [3]:
print("Shape of Images:",np_Images.shape)
print("Shape of Labels:",np_Labels.shape)

Shape of Images: (16785, 224, 224, 3)
Shape of Labels: (16785,)


In [4]:
np.bincount(np_Labels)

array([9955,  229,  766, 2061,  426, 3348])

In [5]:
np_Images=tf.keras.applications.resnet50.preprocess_input(np_Images)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(np_Images, np_Labels, test_size=0.1, random_state=1, stratify=np_Labels)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_tmp, test_size=0.11, random_state=1 ,stratify=y_train_tmp)

In [9]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


Using TensorFlow backend.


In [10]:
IMG_SHAPE = X_train[0].shape

base_model =tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                          include_top=False,
                                                          weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
base_model.trainable = False

In [12]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [13]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    
#     tf.keras.layers.Dense(128, activation=None),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.ReLU(),
    
#     tf.keras.layers.Dense(64, activation=None),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.ReLU(),

    tf.keras.layers.Dense(6, activation='softmax') 
])

In [14]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
X_val.shape, X_train.shape, X_test.shape

((1662, 224, 224, 3), (13444, 224, 224, 3), (1679, 224, 224, 3))

In [15]:
batch_size = 128

# 학습 이미지에 적용한 augmentation 인자를 지정해줍니다.
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# 검증 및 테스트 이미지는 augmentation을 적용하지 않습니다. 모델 성능을 평가할 때에는 이미지 원본을 사용합니다.
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow(
        X_train, y=y_train,  # this is the target directory
        batch_size=batch_size)  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow(
        X_val, y=y_val,
        batch_size=batch_size)

test_generator = test_datagen.flow(
        X_test, y=y_test,
        batch_size=batch_size)

In [21]:
len(train_generator)

106

In [16]:
callback = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=False)]

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        epochs=30,
        callbacks=callback,
        use_multiprocessing=True)
# model.save_weights('first_try.h5')

Epoch 1/30
106/106 [==============================] - 142s 1s/step - loss: 0.6305 - acc: 0.8047 - val_loss: 1.2296 - val_acc: 0.6793
Epoch 2/30
106/106 [==============================] - 135s 1s/step - loss: 0.3272 - acc: 0.9010 - val_loss: 2.0987 - val_acc: 0.6372
Epoch 3/30
106/106 [==============================] - 137s 1s/step - loss: 0.2611 - acc: 0.9177 - val_loss: 2.2152 - val_acc: 0.6348
Epoch 4/30
106/106 [==============================] - 133s 1s/step - loss: 0.2329 - acc: 0.9252 - val_loss: 2.5334 - val_acc: 0.6306
Epoch 5/30
106/106 [==============================] - 136s 1s/step - loss: 0.2102 - acc: 0.9328 - val_loss: 2.2525 - val_acc: 0.6480
Epoch 6/30
106/106 [==============================] - 135s 1s/step - loss: 0.1976 - acc: 0.9368 - val_loss: 2.6177 - val_acc: 0.6288
Epoch 7/30
106/106 [==============================] - 135s 1s/step - loss: 0.1805 - acc: 0.9435 - val_loss: 2.6142 - val_acc: 0.6366
Epoch 8/30
106/106 [==============================] - 136s 1s/step - 

Exception in thread Thread-85:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 752, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 731, in pool_fn
    initargs=(seqs, None, get_worker_id_queue()))
  File "/opt/anaconda3/lib/python3.7/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 176, in __init__
    self._repopulate_pool()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 241, in _repopulate_pool
    w.start()
  File "/opt/anaconda3/lib

Epoch 9/30
 89/106 [========================>.....] - ETA: 21s - loss: 0.1672 - acc: 0.9469